Первая версия - 6 эпох lb = 0.01918

Вторая версия - 20 эпох + GPU сделал lb = 0.02666 (плохо!)

Третья версия - DropOut + Sigmoid

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
dir0 = '../input/lish-moa/'
# dir0 = ""
train_features = pd.read_csv(dir0 + 'train_features.csv')
train_targets_scored = pd.read_csv(dir0 + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(dir0 + 'train_targets_nonscored.csv')
sample_submission = pd.read_csv(dir0 + 'sample_submission.csv')
test_features = pd.read_csv(dir0 + 'test_features.csv')

In [ ]:
def make_features(train_features):
    train_features['cp_type'] = (train_features['cp_type'] == 'ctl_vehicle').astype(int)
    train_features['cp_time'] = (train_features['cp_time'] - 48) / 24
    train_features['cp_dose'] = (train_features['cp_dose'] == 'D1').astype(int)
    train_features_ids = train_features.pop('sig_id')
    return (train_features, train_features_ids)

In [ ]:
train_features, train_features_ids = make_features(train_features)
test_features, test_features_ids = make_features(test_features)

train_targets_scored_sig_id = train_targets_scored.pop('sig_id')

print (train_features.shape, test_features.shape)

In [ ]:
ii = train_features.cp_type.values==0
train_features = train_features.loc[ii]
train_targets_scored = train_targets_scored.loc[ii]
print (train_features.shape, test_features.shape)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
train_targets_scored.head()

In [ ]:
train_dataset = TensorDataset(torch.tensor(train_features.values, dtype=torch.float32).to(device),
                              torch.tensor(train_targets_scored.values, dtype=torch.float32).to(device))
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [ ]:
train_features.shape, train_targets_scored.shape

In [ ]:
class MyModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.model = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Dropout(0.3),
            nn.Linear(in_features, 800),
            nn.Sigmoid(),

            nn.BatchNorm1d(800),            # nn.Dropout(0.1),
            nn.Dropout(0.3),
            nn.Linear(800, 600),
            nn.Sigmoid(),

            nn.BatchNorm1d(600),            # nn.Dropout(0.1),
            nn.Dropout(0.3),
            nn.Linear(600, 400),
            nn.Sigmoid(),

            nn.BatchNorm1d(400),
            nn.Linear(400, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
    
# class MyModel(nn.Module):
#     def __init__(self, in_features, out_features):
#         super().__init__()
#         self.in_features = in_features
#         self.out_features = out_features

#         self.model = nn.Sequential(
#             nn.BatchNorm1d(in_features),
#             nn.Linear(in_features, 800),
#             nn.ReLU(),

#             nn.BatchNorm1d(800),            # nn.Dropout(0.1),
#             nn.Linear(800, 600),
#             nn.ReLU(),

#             nn.BatchNorm1d(600),            # nn.Dropout(0.1),
#             nn.Linear(600, 400),
#             nn.ReLU(),

#             nn.BatchNorm1d(400),
#             nn.Linear(400, out_features),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         return self.model(x)

In [ ]:
%%time
model = MyModel(train_features.shape[1], train_targets_scored.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) #SGD(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 0.05, 10)
criterion = nn.BCELoss()
max_epoch = 20

for epoch in range(max_epoch):
    model.train()
    for i, (x_batch, y_batch) in enumerate(train_dataloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        preds = model(x_batch)

        optimizer.zero_grad()
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        if i % 20 == 0:
            print(f'Epoch: {epoch}, train loss: {loss.item():12.5f}')

    model.eval()
    with torch.no_grad():
        train, y_train = train_dataset.tensors
        train, y_train = train.to(device), y_train.to(device)
        train_preds = model(train)
        train_loss = criterion(train_preds, y_train).item()

In [ ]:
# model = MyModel(train_features.shape[1], train_targets_scored.shape[1])
# optimizer = torch.optim.Adam(model.parameters()) #SGD(model.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 0.05, 10)
# criterion = nn.BCELoss()
# max_epoch = 6

# for epoch in range(max_epoch):
#     model.train()
#     for i, (x_batch, y_batch) in enumerate(train_dataloader):
#         preds = model(x_batch)

#         optimizer.zero_grad()
#         loss = criterion(preds, y_batch)
#         loss.backward()
#         optimizer.step()

#         if i % 20 == 0:
#             print(f'Epoch: {epoch}, train loss: {loss.item():12.5f}')

#     model.eval()
#     with torch.no_grad():
#         train, y_train = train_dataset.tensors
#         train_preds = model(train)
#         train_loss = criterion(train_preds, y_train).item()
        
# #         test, y_test = test_dataset.tensors
# #         test_preds = model(test)
# #         test_loss = criterion(test_preds, y_test).item()
#         print(f'Epoch {epoch} final: train loss: {train_loss}')#, f'test loss: {test_loss}')

In [ ]:
model.eval()
with torch.no_grad():
    probs = model(torch.tensor(test_features.values, dtype=torch.float32).to(device))
    
probs = probs.to("cpu").numpy()

In [ ]:
ii2 = test_features.cp_type.values==1 # 1 !!!!
probs[ii2,:] = 0.0

In [ ]:
df = pd.DataFrame(probs, index=test_features_ids, columns=train_targets_scored.columns)

In [ ]:
df.to_csv('submission.csv')